# 🤖 Kapitola 7: AI v každodenním životě

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 1 | AI pro začátečníky</h2>
    <p style="color: white; margin: 10px 0;">🎯 Praktická kapitola - Doporučovací systémy a AI asistenti</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na:

- **Doporučovací systémy** - jak Netflix, Spotify a YouTube "vědí", co chcete
- **Kolaborativní filtrování** - najdeme vaše "digitální dvojče"
- **Kosinová podobnost** - matematika za doporučeními
- **Hlasoví asistenti** - jak Siri a Alexa rozumí vaší řeči
- **AI v medicíně a dopravě** - jak AI zachraňuje životy

## ⚠️ Předpoklady
Tato kapitola navazuje na: Kapitoly 1-6 (základy AI, etika)

---

## 🔧 Instalace knihoven

Tento notebook je navržen pro **Google Colab**, kde jsou knihovny již předinstalované.

In [ ]:
# Instalace knihoven (v Colabu obvykle není potřeba)
# !pip install pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Nastavení vizualizací
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("="*60)
print("📚 KAPITOLA 7: AI v každodenním životě")
print("="*60)
print(f"📅 Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("✅ Všechny knihovny načteny!")
print("="*60)

---

## 📖 Část 1: Neviditelní AI pomocníci kolem nás

Váš den je plný interakcí s umělou inteligencí:

| Čas | Aktivita | AI na pozadí |
|-----|----------|-------------|
| 7:00 | Budík zahraje hudbu | Spotify doporučil playlist |
| 7:30 | Navigace do práce | Waze předpovídá zácpy |
| 12:00 | Prohlížení Instagramu | Algoritmus řadí příspěvky |
| 18:00 | Email od spammera | Gmail ho automaticky zablokoval |
| 21:00 | Výběr filmu | Netflix doporučuje |

Pojďme prozkoumat, jak tito **digitální pomocníci** fungují!

In [ ]:
# 📊 Vizualizace: Kolik AI interakcí máme denně?

interakce = {
    'Doporučení obsahu\n(Netflix, YouTube, Spotify)': 45,
    'Vyhledávání\n(Google, Bing)': 25,
    'Sociální sítě\n(Feed algoritmy)': 60,
    'Spam filtry\n(Email)': 15,
    'Navigace\n(GPS, mapy)': 8,
    'Hlasoví asistenti\n(Siri, Alexa)': 5,
    'Autocomplete\n(Klávesnice)': 100
}

fig, ax = plt.subplots(figsize=(12, 7))

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(interakce)))
bars = ax.barh(list(interakce.keys()), list(interakce.values()), color=colors)

ax.set_xlabel('Průměrný počet interakcí denně', fontsize=12)
ax.set_title('🤖 Kolik AI rozhodnutí ovlivní váš den?', fontsize=14, fontweight='bold')

# Přidáme hodnoty na konce sloupců
for bar, value in zip(bars, interakce.values()):
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
            f'{value}x', va='center', fontsize=11)

ax.set_xlim(0, 120)
plt.tight_layout()
plt.show()

print(f"\n📊 CELKEM: Přibližně {sum(interakce.values())} AI rozhodnutí denně!")
print("\n💡 A to jsme nezapočítali tisíce dalších AI systémů na pozadí...")

---

## 📖 Část 2: Doporučovací systémy - Jak Netflix "ví", co chcete

### Princip kolaborativního filtrování

**Analogie s přáteli:**
> Představte si kamaráda Petra, který má úplně stejný filmový vkus jako vy. 
> Všechno, co se líbilo jemu, se líbilo i vám. 
> Když Petr řekne: "Včera jsem viděl skvělý film!", co uděláte? 
> Pravděpodobně si ho pustíte.

**Jak to dělá AI:**
1. Najde tisíce uživatelů s podobným vkusem jako vy
2. Podívá se, co se těmto "digitálním dvojčatům" líbilo
3. Doporučí vám filmy, které jste ještě neviděli

Je to **matematicky řízená šeptanda** ve velkém měřítku!

In [ ]:
# 🎬 Vytvoříme filmovou databázi

# Hodnocení filmů (1-5 hvězdiček, 0 = neviděl)
filmova_data = {
    'Uživatel': ['Anna', 'Pavel', 'Jana', 'Tomáš', 'Eva', 'Petr', 'VY'],
    'Matrix': [5, 5, 2, 5, 4, 5, 5],
    'Pán Prstenů': [5, 4, 2, 5, 5, 4, 5],
    'Titanic': [2, 1, 5, 1, 3, 2, 2],
    'Avengers': [4, 5, 3, 4, 4, 5, 4],
    'Forrest Gump': [4, 3, 5, 3, 5, 3, 0],  # 0 = neviděli jste
    'Počátek': [5, 5, 2, 5, 4, 5, 0],       # 0 = neviděli jste
    'Frozen': [2, 1, 5, 1, 3, 1, 0],        # 0 = neviděli jste
    'Duna': [5, 4, 2, 5, 4, 4, 0]           # 0 = neviděli jste
}

df = pd.DataFrame(filmova_data).set_index('Uživatel')

print("🎬 FILMOVÁ DATABÁZE HODNOCENÍ")
print("="*60)
print("\n(Hodnocení 1-5 hvězdiček, 0 = film nebyl viděn)\n")
display(df.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=5))

In [ ]:
# 🔍 Vizualizace hodnocení jako heatmapu

fig, ax = plt.subplots(figsize=(12, 6))

# Vytvoříme masku pro neviděné filmy (0)
mask = df == 0

sns.heatmap(df, annot=True, cmap='RdYlGn', center=3, 
            linewidths=0.5, ax=ax, fmt='d',
            cbar_kws={'label': 'Hodnocení'})

ax.set_title('🎬 Matice hodnocení filmů', fontsize=14, fontweight='bold')
ax.set_xlabel('Filmy')
ax.set_ylabel('Uživatelé')

# Označíme VY řádek
ax.axhline(y=6, color='blue', linewidth=3)
ax.axhline(y=7, color='blue', linewidth=3)

plt.tight_layout()
plt.show()

print("\n💡 Modrý rámeček označuje VAŠE hodnocení.")
print("   Hodnoty 0 znamenají filmy, které jste ještě neviděli.")

---

## 📖 Část 3: Kosinová podobnost - Matematika za doporučeními

### Co je kosinová podobnost?

Představte si, že každý uživatel je **šipka (vektor)** v prostoru:
- Směr šipky = filmový vkus
- Pokud dvě šipky míří **stejným směrem**, uživatelé mají **podobný vkus**

**Kosinová podobnost** měří úhel mezi těmito šipkami:
- **1.0** = Identický vkus (šipky míří stejně)
- **0.0** = Žádná podobnost (šipky jsou kolmé)
- **-1.0** = Opačný vkus (šipky míří opačně)

### Vzorec:
$$\text{podobnost}(A, B) = \frac{A \cdot B}{||A|| \times ||B||}$$

In [ ]:
# 🧮 Implementace kosinové podobnosti

def spocitej_podobnost(uzivatel1, uzivatel2, data):
    """
    Spočítá kosinovou podobnost mezi dvěma uživateli.
    Bere v úvahu pouze filmy, které viděli OBA uživatelé.
    """
    # Získáme hodnocení obou uživatelů
    hodnoceni1 = data.loc[uzivatel1]
    hodnoceni2 = data.loc[uzivatel2]
    
    # Najdeme filmy, které viděli OBA (hodnocení > 0)
    spolecne = (hodnoceni1 > 0) & (hodnoceni2 > 0)
    
    if spolecne.sum() == 0:
        return 0.0  # Žádné společné filmy
    
    # Vezmeme pouze společná hodnocení
    vec1 = hodnoceni1[spolecne].values.reshape(1, -1)
    vec2 = hodnoceni2[spolecne].values.reshape(1, -1)
    
    # Kosinová podobnost
    podobnost = cosine_similarity(vec1, vec2)[0][0]
    
    return podobnost

# Spočítáme podobnost VY s ostatními
print("🔍 HLEDÁME VAŠE DIGITÁLNÍ DVOJČE")
print("="*50)
print("\nKosinová podobnost s ostatními uživateli:\n")

podobnosti = {}
for uzivatel in df.index:
    if uzivatel != 'VY':
        podobnost = spocitej_podobnost('VY', uzivatel, df)
        podobnosti[uzivatel] = podobnost
        # Vizuální indikátor podobnosti
        hvezdicky = '⭐' * int(podobnost * 5)
        print(f"  {uzivatel:10} → {podobnost:.3f} {hvezdicky}")

# Najdeme nejpodobnějšího uživatele
digitalni_dvojce = max(podobnosti, key=podobnosti.get)
max_podobnost = podobnosti[digitalni_dvojce]

print("\n" + "="*50)
print(f"\n🎉 Vaše digitální dvojče je: {digitalni_dvojce}")
print(f"   Podobnost vkusu: {max_podobnost:.1%}")

In [ ]:
# 📊 Vizualizace podobností

fig, ax = plt.subplots(figsize=(10, 6))

uzivatele = list(podobnosti.keys())
hodnoty = list(podobnosti.values())

colors = ['green' if v == max(hodnoty) else 'steelblue' for v in hodnoty]

bars = ax.bar(uzivatele, hodnoty, color=colors, edgecolor='black')

ax.set_ylabel('Kosinová podobnost', fontsize=12)
ax.set_xlabel('Uživatelé', fontsize=12)
ax.set_title('🔍 Podobnost vašeho filmového vkusu s ostatními', 
             fontsize=14, fontweight='bold')
ax.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Práh vysoké podobnosti')
ax.set_ylim(0, 1.1)

# Přidáme hodnoty nad sloupce
for bar, hodnota in zip(bars, hodnoty):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{hodnota:.2f}', ha='center', va='bottom', fontsize=11)

ax.legend()
plt.tight_layout()
plt.show()

print(f"\n💚 Zelený sloupec = Vaše digitální dvojče ({digitalni_dvojce})")

---

## 📖 Část 4: Doporučení filmů

Teď použijeme naše "digitální dvojče" k doporučení filmů, které jste ještě neviděli!

In [ ]:
# 🎬 Doporučíme filmy na základě digitálního dvojčete

def doporuc_filmy(uzivatel, data, podobnosti, top_n=3):
    """
    Doporučí filmy na základě hodnocení podobných uživatelů.
    """
    # Najdeme filmy, které uživatel ještě neviděl
    nevidene = data.loc[uzivatel][data.loc[uzivatel] == 0].index.tolist()
    
    if not nevidene:
        return []
    
    # Spočítáme skóre pro každý neviděný film
    skore_filmu = {}
    
    for film in nevidene:
        vazeny_soucet = 0
        soucet_vah = 0
        
        for jiny_uzivatel, podobnost in podobnosti.items():
            hodnoceni = data.loc[jiny_uzivatel, film]
            if hodnoceni > 0:  # Pokud tento uživatel film viděl
                vazeny_soucet += podobnost * hodnoceni
                soucet_vah += podobnost
        
        if soucet_vah > 0:
            skore_filmu[film] = vazeny_soucet / soucet_vah
    
    # Seřadíme podle skóre a vrátíme top N
    serazene = sorted(skore_filmu.items(), key=lambda x: x[1], reverse=True)
    return serazene[:top_n]

# Získáme doporučení
doporuceni = doporuc_filmy('VY', df, podobnosti)

print("🎬 PERSONALIZOVANÁ DOPORUČENÍ PRO VÁS")
print("="*50)
print("\nNa základě vašeho vkusu vám doporučujeme:\n")

for i, (film, skore) in enumerate(doporuceni, 1):
    hvezdicky = '⭐' * round(skore)
    print(f"  {i}. {film}")
    print(f"     Předpovězené hodnocení: {skore:.1f}/5 {hvezdicky}")
    print()

In [ ]:
# 📊 Vizualizace doporučení

fig, ax = plt.subplots(figsize=(10, 6))

filmy = [d[0] for d in doporuceni]
skore = [d[1] for d in doporuceni]

colors = plt.cm.RdYlGn(np.array(skore) / 5)

bars = ax.barh(filmy, skore, color=colors, edgecolor='black', height=0.6)

ax.set_xlabel('Předpovězené hodnocení', fontsize=12)
ax.set_title('🎬 Top doporučené filmy pro VÁS', fontsize=14, fontweight='bold')
ax.set_xlim(0, 5.5)

# Přidáme hodnocení a hvězdičky
for bar, s in zip(bars, skore):
    hvezdicky = '⭐' * round(s)
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
            f'{s:.1f} {hvezdicky}', va='center', fontsize=11)

# Přidáme vertikální čáry pro hodnocení
for i in range(1, 6):
    ax.axvline(x=i, color='gray', linestyle=':', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Tyto filmy doporučujeme, protože se líbily uživatelům")
print("   s podobným filmovým vkusem jako vy!")

---

## 📖 Část 5: Hlasoví asistenti - Od zvuku k akci

Když řeknete **"Hej Siri, jaké bude počasí?"**, spustí se fascinující řetězec:

### Pipeline hlasového asistenta:

```
🎤 Zvuk → 📝 Text → 🧠 Porozumění → ⚡ Akce → 🔊 Odpověď
```

1. **Speech-to-Text (STT):** Zvuková vlna → "jaké bude počasí"
2. **Natural Language Understanding (NLU):** Identifikace záměru (intent)
3. **Provedení akce:** Dotaz do API počasí
4. **Text-to-Speech (TTS):** "Dnes bude slunečno, 22 stupňů" → zvuk

In [ ]:
# 🎤 Vizualizace pipeline hlasového asistenta

import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(14, 8))

# Definice kroků pipeline
kroky = [
    ('🎤', 'Zvuková\nvlna', 'Mikrofon zachytí\nváš hlas', '#E3F2FD'),
    ('📝', 'Speech-to-Text', 'AI převede zvuk\nna text', '#E8F5E9'),
    ('🧠', 'NLU\n(Porozumění)', 'AI pochopí\nváš záměr', '#FFF3E0'),
    ('⚡', 'Akce', 'Získání dat\nz API/databáze', '#FCE4EC'),
    ('🔊', 'Text-to-Speech', 'AI vytvoří\nhlasovou odpověď', '#F3E5F5')
]

# Kreslíme boxy
y_pos = 0.5
box_width = 0.15
box_height = 0.35
spacing = 0.17

for i, (emoji, nazev, popis, barva) in enumerate(kroky):
    x_pos = 0.08 + i * spacing
    
    # Box
    rect = mpatches.FancyBboxPatch((x_pos, y_pos - box_height/2), 
                                    box_width, box_height,
                                    boxstyle="round,pad=0.02",
                                    facecolor=barva, edgecolor='gray',
                                    linewidth=2)
    ax.add_patch(rect)
    
    # Emoji nahoře
    ax.text(x_pos + box_width/2, y_pos + 0.12, emoji, 
            fontsize=24, ha='center', va='center')
    
    # Název
    ax.text(x_pos + box_width/2, y_pos, nazev, 
            fontsize=9, ha='center', va='center', fontweight='bold')
    
    # Popis pod boxem
    ax.text(x_pos + box_width/2, y_pos - 0.25, popis, 
            fontsize=8, ha='center', va='top', style='italic')
    
    # Šipka (kromě posledního)
    if i < len(kroky) - 1:
        ax.annotate('', xy=(x_pos + spacing, y_pos),
                   xytext=(x_pos + box_width + 0.01, y_pos),
                   arrowprops=dict(arrowstyle='->', color='#333', lw=2))

# Příklad dotazu
ax.text(0.5, 0.92, '"Hej Siri, jaké bude počasí?"', 
        fontsize=14, ha='center', va='center',
        bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange'))

# Příklad odpovědi
ax.text(0.5, 0.08, '"Dnes bude slunečno, 22 stupňů Celsia."', 
        fontsize=14, ha='center', va='center',
        bbox=dict(boxstyle='round', facecolor='lightgreen', edgecolor='green'))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('🎤 Pipeline hlasového asistenta', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\n⏱️ Celý tento proces trvá méně než 1 sekundu!")

In [ ]:
# 🧠 Simulace NLU - rozpoznání záměru (intent)

def rozpoznej_zamer(text):
    """
    Jednoduchá simulace NLU - rozpoznání záměru z textu.
    V reálu se používají pokročilé ML modely.
    """
    text = text.lower()
    
    # Slovník klíčových slov a záměrů
    zamery = {
        'počasí': {'intent': 'WEATHER', 'action': 'Dotaz na API počasí'},
        'čas': {'intent': 'TIME', 'action': 'Získání systémového času'},
        'budík': {'intent': 'ALARM', 'action': 'Nastavení alarmu'},
        'připomínka': {'intent': 'REMINDER', 'action': 'Vytvoření připomínky'},
        'přehraj': {'intent': 'PLAY_MUSIC', 'action': 'Spuštění přehrávače'},
        'hudba': {'intent': 'PLAY_MUSIC', 'action': 'Spuštění přehrávače'},
        'zavolej': {'intent': 'CALL', 'action': 'Vytočení kontaktu'},
        'zpráv': {'intent': 'MESSAGE', 'action': 'Otevření zpráv'},
        'naviguj': {'intent': 'NAVIGATION', 'action': 'Spuštění navigace'},
        'cesta': {'intent': 'NAVIGATION', 'action': 'Spuštění navigace'}
    }
    
    for klicove_slovo, info in zamery.items():
        if klicove_slovo in text:
            return info
    
    return {'intent': 'UNKNOWN', 'action': 'Vyhledání na internetu'}

# Testovací dotazy
dotazy = [
    "Jaké bude zítra počasí?",
    "Nastav budík na 7 ráno",
    "Přehraj moji oblíbenou hudbu",
    "Zavolej mamince",
    "Naviguj mě domů",
    "Kolik je hodin?"
]

print("🧠 SIMULACE NLU - ROZPOZNÁNÍ ZÁMĚRU")
print("="*60)
print()

for dotaz in dotazy:
    vysledek = rozpoznej_zamer(dotaz)
    print(f"📝 Dotaz: \"{dotaz}\"")
    print(f"   🎯 Intent: {vysledek['intent']}")
    print(f"   ⚡ Akce: {vysledek['action']}")
    print()

---

## 📖 Část 6: AI v medicíně a dopravě

### 🏥 AI v medicíně
- Analýza rentgenových snímků a MRI
- Detekce nádorů, které lidské oko přehlédne
- AI jako "druhý pár očí" pro lékaře

### 🚗 AI v dopravě
- Predikce dopravních zácp v reálném čase
- Dynamické přesměrování řidičů
- Autonomní vozidla

In [ ]:
# 📊 Porovnání přesnosti AI vs. člověk v diagnostice

oblasti = ['Rentgen\nplic', 'Kožní\nléze', 'Oční\nsítnice', 'Mamografie', 'CT\nmozku']

presnost_lekar = [87, 83, 91, 85, 88]
presnost_ai = [94, 95, 96, 91, 93]

x = np.arange(len(oblasti))
sirka = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - sirka/2, presnost_lekar, sirka, label='Zkušený lékař', 
               color='#3498db', edgecolor='black')
bars2 = ax.bar(x + sirka/2, presnost_ai, sirka, label='AI asistent', 
               color='#2ecc71', edgecolor='black')

ax.set_ylabel('Přesnost diagnostiky (%)', fontsize=12)
ax.set_title('🏥 AI jako pomocník v lékařské diagnostice', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(oblasti)
ax.legend()
ax.set_ylim(70, 100)

# Přidáme hodnoty
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{int(bar.get_height())}%', ha='center', va='bottom', fontsize=10)

for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{int(bar.get_height())}%', ha='center', va='bottom', fontsize=10)

ax.axhline(y=90, color='red', linestyle='--', alpha=0.5, label='Cíl: 90%')

plt.tight_layout()
plt.show()

print("\n💡 AI NEnahrazuje lékaře, ale pomáhá jako 'druhý pár očí'!")
print("   Kombinace člověk + AI dosahuje nejlepších výsledků.")

In [ ]:
# 🚗 Simulace dopravní predikce

np.random.seed(42)

# Simulace hodinového provozu
hodiny = list(range(24))

# Typický vzor provozu (špička ráno a večer)
zakladni_provoz = np.array([
    10, 8, 5, 5, 8, 20,  # 0-5
    45, 85, 95, 70, 55, 60,  # 6-11 (ranní špička)
    65, 60, 55, 60, 75, 95,  # 12-17 (odpolední špička)
    80, 55, 40, 30, 20, 15   # 18-23
])

# AI predikce (s malou chybou)
ai_predikce = zakladni_provoz + np.random.normal(0, 5, 24)
ai_predikce = np.clip(ai_predikce, 0, 100)

# Skutečný provoz (s větší variabilitou)
skutecny = zakladni_provoz + np.random.normal(0, 8, 24)
skutecny = np.clip(skutecny, 0, 100)

fig, ax = plt.subplots(figsize=(14, 6))

ax.fill_between(hodiny, skutecny, alpha=0.3, color='red', label='Skutečný provoz')
ax.plot(hodiny, skutecny, 'r-', linewidth=2, marker='o', markersize=5)

ax.plot(hodiny, ai_predikce, 'g--', linewidth=2, marker='s', 
        markersize=5, label='AI predikce (Waze/Google Maps)')

ax.set_xlabel('Hodina', fontsize=12)
ax.set_ylabel('Intenzita provozu (%)', fontsize=12)
ax.set_title('🚗 AI predikce dopravního provozu v reálném čase', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.set_xticks(hodiny)
ax.set_xlim(0, 23)
ax.set_ylim(0, 110)

# Označení špiček
ax.axvspan(7, 9, alpha=0.2, color='orange', label='Ranní špička')
ax.axvspan(16, 18, alpha=0.2, color='orange', label='Odpolední špička')

ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Výpočet přesnosti predikce
mae = np.mean(np.abs(skutecny - ai_predikce))
print(f"\n📊 Průměrná chyba AI predikce: {mae:.1f}%")
print("\n💡 Díky AI predikci může navigace přesměrovat řidiče")
print("   a rozložit provoz DŘÍVE než vznikne zácpa!")

---

## 🎯 Mini-projekt: Vlastní doporučovací systém

Vytvořte doporučovací systém pro **hudbu** nebo **knihy**!

### Zadání:
1. Vytvořte databázi alespoň 6 uživatelů a 8 položek (písniček/knih)
2. Implementujte kolaborativní filtrování
3. Najděte "digitální dvojče" pro nového uživatele
4. Doporučte 3 položky

In [ ]:
# 🎵 MINI-PROJEKT: Doporučovací systém pro hudbu
# Doplňte svá vlastní data a implementaci!

# Příklad struktury dat (upravte podle sebe):
hudebni_data = {
    'Uživatel': ['Alice', 'Bob', 'Carol', 'David', 'Eve', 'VY'],
    'Ed Sheeran': [5, 3, 5, 2, 4, 4],
    'Metallica': [2, 5, 1, 5, 3, 4],
    'Taylor Swift': [5, 2, 5, 1, 4, 0],  # 0 = neposlouchali jste
    'AC/DC': [1, 5, 2, 5, 2, 0],
    'Billie Eilish': [4, 2, 4, 2, 5, 0],
    'Queen': [4, 5, 4, 5, 4, 5],
    'Adele': [5, 2, 5, 2, 4, 0],
    'Linkin Park': [3, 5, 2, 5, 3, 5]
}

df_hudba = pd.DataFrame(hudebni_data).set_index('Uživatel')

print("🎵 HUDEBNÍ DATABÁZE")
print("="*60)
display(df_hudba.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=5))

# TODO: Implementujte kolaborativní filtrování pro hudbu
# Hint: Použijte stejné funkce jako výše (spocitej_podobnost, doporuc_filmy)

print("\n🎯 ÚKOL: Implementujte doporučení hudby!")
print("   Kdo je vaše hudební 'dvojče'?")
print("   Které interprety byste měli zkusit?")

In [ ]:
# 💡 ŘEŠENÍ mini-projektu (odkryjte až po vlastním pokusu)

# Spočítáme podobnosti pro hudbu
podobnosti_hudba = {}
for uzivatel in df_hudba.index:
    if uzivatel != 'VY':
        podobnost = spocitej_podobnost('VY', uzivatel, df_hudba)
        podobnosti_hudba[uzivatel] = podobnost

print("🎵 VÝSLEDKY HUDEBNÍHO DOPORUČOVACÍHO SYSTÉMU")
print("="*60)

# Hudební dvojče
dvojce_hudba = max(podobnosti_hudba, key=podobnosti_hudba.get)
print(f"\n🎯 Vaše hudební dvojče: {dvojce_hudba}")
print(f"   Podobnost: {podobnosti_hudba[dvojce_hudba]:.1%}")

# Doporučení
doporuceni_hudba = doporuc_filmy('VY', df_hudba, podobnosti_hudba)

print("\n🎵 Doporučení interpreti:")
for i, (interpret, skore) in enumerate(doporuceni_hudba, 1):
    print(f"   {i}. {interpret} (předpověď: {skore:.1f}/5 ⭐)")

---

## 🧪 Cvičení

### Cvičení 1: AI detektiv
Zkuste být jeden den "AI detektivem" - zapisujte si všechny AI interakce, které zaznamenáte.

### Cvičení 2: Rozšíření doporučovacího systému
Přidejte do filmové databáze:
- Více uživatelů
- Více filmů
- Žánrové tagy

### Cvičení 3: NLU rozšíření
Rozšiřte funkci `rozpoznej_zamer()` o další intenty a klíčová slova.

In [ ]:
# 🧪 Prostor pro vaše experimenty

# Zde můžete experimentovat s kódem z kapitoly
# Zkuste přidat nové filmy, uživatele, nebo vytvořit vlastní doporučovací systém!



---

## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ AI je všudypřítomná v každodenním životě</li>
        <li>✓ Doporučovací systémy používají kolaborativní filtrování</li>
        <li>✓ Kosinová podobnost měří shodu vkusu uživatelů</li>
        <li>✓ Hlasoví asistenti používají pipeline: STT → NLU → Akce → TTS</li>
        <li>✓ AI pomáhá v medicíně a dopravě jako "druhý pár očí"</li>
    </ul>
    
    <h3 style="color: white;">🔑 Klíčové pojmy</h3>
    <ul>
        <li><b>Kolaborativní filtrování</b> - doporučení na základě podobných uživatelů</li>
        <li><b>Kosinová podobnost</b> - měření úhlu mezi vektory preferencí</li>
        <li><b>NLU</b> - Natural Language Understanding (porozumění přirozenému jazyku)</li>
        <li><b>Intent</b> - záměr uživatele v hlasovém příkazu</li>
    </ul>
    
    <h3 style="color: white;">➡️ Další kapitola</h3>
    <p>V kapitole 8 se podíváme na <b>AI ve hrách</b> a algoritmus Minimax!</p>
</div>

---

*📅 Notebook vytvořen: 2025*  
*🎓 Kurz: AI pro každého - Od základů k praxi*  
*🔧 Prostředí: Google Colab (Python 3)*